In [11]:
!pip install confounds
!pip install cca-zoo --upgrade

You should consider upgrading via the 'c:\users\chapm\pycharmprojects\confounds\venv\scripts\python.exe -m pip install --upgrade pip' command.


  Using cached cca_zoo-1.10.6-py3-none-any.whl (80 kB)
  Using cached scikit_learn-1.0.1-cp39-cp39-win_amd64.whl (7.2 MB)
  Using cached mvlearn-0.4.1-py3-none-any.whl (2.1 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.6.3
    Uninstalling scipy-1.6.3:
      Successfully uninstalled scipy-1.6.3


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\chapm\\PycharmProjects\\confounds\\venv\\Lib\\site-packages\\~cipy\\.libs\\libansari.R6EA3HQP5KZ6TAXU4Y4ZVTRPT7UVA53Z.gfortran-win_amd64.dll'
Check the permissions.

You should consider upgrading via the 'c:\users\chapm\pycharmprojects\confounds\venv\scripts\python.exe -m pip install --upgrade pip' command.


# Tutorial using confounds library with Canonical Correlation Analysis

In [12]:
#imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from confounds import Residualize
from cca_zoo.models import CCA

In [13]:
#parameters
np.random.seed(42)
#samples
n = 1000
#X features
p = 10
#Y features
q = 10
#Number of scanners
s=2

## Generate Simulated Data
We use a latent variable model to generate 2 views of data. One latent variable represents a binary confound variable such as the scanner type in a neuroimaging study. The other latent variable represents some underlying relationship between the 2 views. Notice that X and Y are multivariate and CCA will look to identify multivariate associations.

In [14]:
# Make a binary confounding variable such as scanner type/gender
confound = np.random.randint(s, size=(n, 1))

# Generate the underlying relationship of interest as a latent variable
relationship_of_interest = np.random.normal(0, 1, size=(n, 1))

# Random weights representing the effect of the confounding variable on the data
weights_cx = np.random.normal(0, 1, size=(1, p))
weights_cy = np.random.normal(0, 1, size=(1, q))

# Random weights representing the relationship of interest in the data
weights_rx = np.random.normal(0, 1, size=(1, p))
weights_ry = np.random.normal(0, 1, size=(1, q))

# Generate the 'observed' data views by projecting from the latent variables to the feature space using these weights
X = np.dot(confound, weights_cx) + np.dot(relationship_of_interest, weights_rx) + np.random.normal(0, 1, size=(n, p))
Y = np.dot(confound, weights_cy) + np.dot(relationship_of_interest, weights_cy) + np.random.normal(0, 1, size=(n, q))

## CCA using the confounded data
First we fit a CCA model to the original data. By plotting the learnt projections to the latent space, we see that the learnt association is dominated by the effect of the confounding variable (which could be for example scanner type).

In [16]:
cca = CCA().fit((X, Y))
transformed_x, transformed_y = cca.transform((X,Y))

data = pd.DataFrame(
    {'Scanner Type': np.squeeze(confound).astype(str), 'view 1 projection': np.squeeze(transformed_x),
     'view 2 projection': np.squeeze(transformed_y)})
cca_pp = sns.pairplot(data, hue='Scanner Type')
cca_pp.fig.suptitle('Confounded CCA')

ValueError: number of views passed should match number of parameter clen(views)=1 and len(c)=2

## CCA using the deconfounded data
Now we use confounds to remove the variance associated with the scanner type from **both** views. To do this we can use two instances of Residualize() - one for each view (X and Y). This time, the CCA model finds a similarly strong relationship between the two views but this association is not driven by the 'scanner type' confounding variable.

In [ ]:
# We can use Residualize() to regress out the effect of the confounding variable
resid_x = Residualize()
resid_y = Residualize()
resid_x.fit(X, confound)
resid_y.fit(Y, confound)
deconf_X = resid_x.transform(X, confound)
deconf_Y = resid_y.transform(Y, confound)

# Fit a CCA to the deconfounded data
deconf_cca = CCA().fit((deconf_X, deconf_Y))
deconf_transformed_x, deconf_transform_y = deconf_cca.transform((deconf_X,deconf_Y))

deconf_data = pd.DataFrame(
    {'Scanner Type': np.squeeze(confound).astype(str),
     'view 1 projection': np.squeeze(deconf_transformed_x),
     'view 2 projection': np.squeeze(deconf_transformed_y)})

cca_pp_deconf = sns.pairplot(deconf_data, hue='Scanner Type')
cca_pp_deconf.fig.suptitle('Deconfounded CCA')
plt.show()